# Neural Network

## 딥러닝 맛보기

- 추후 사용할 RNN을 위하여 딥러닝 즉 신경망에 대해 맛보기하는 시간을 가저보자
- 이전 다른 분류기도 그랬지만 분류와 회귀 모두에서 사용할 수 있다.
- 수학적 원리는 정확히 설명하고 구현하기 어려우므로 다른 자료를 참조하기를 권장함
- 기본 구조 외에 다양한 architecture에 대한 것은 [여기](https://github.com/young31/Deep-Learning/tree/master/Architecture)
  
    
- keras를 사용하여 구현할 예정
    - keras는 쉽게 사용하게 해주는 툴
    - tensorflow 등을 백으로 가질 수 있음
    
- 순차모형과 이름을 달아 연결하는 방법이 있음
    - 순차모형이 시작하기는 쉬움
    - 나중에 복잡한 망을 짜려면 개별 망에 연결하는 방법을 사용
    
- 신경망은 greedy하게 해를 찾는 경사하강법을 사용해서 답을 찾아가므로 정규화를 무조건 해줘야함!!!
    - 해당 절에서는 편의를 위해 카테고리변수를 따로 설정하지 않음

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = pd.read_csv('../1.clustering/titanic.csv')

target = data['Survived']
data = data.drop(['PassengerId', 'Survived'], axis=1)

def sex(a):
    if a == 'male':
        return 0
    else:
        return 1
    
def emb(a):
    if a == 'S':
        return 0
    elif a == 'Q':
        return 1
    else:
        return 2
    
data['Sex'] = data['Sex'].map(sex)
data['Embarked'] = data['Embarked'].map(emb)

train_X, test_X, train_y, test_y = train_test_split(data, target, test_size=0.3, random_state=2019, shuffle=True)

In [2]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
tt = std.fit_transform(train_X)
te = std.fit_transform(test_X)

In [3]:
## 층을 만든다
from keras import models, layers, optimizers
import keras.backend as K
K.clear_session()

input_layer = layers.Input(shape=train_X.shape[1:])

layer1 = layers.Dense(32, activation='relu')(input_layer)
layer2 = layers.Dense(16, activation='relu')(layer1)

out_layer = layers.Dense(1, activation='sigmoid')(layer2)

model = models.Model(input_layer, out_layer)

Using TensorFlow backend.


In [4]:
## compile 한다.

model.compile(optimizer = optimizers.Adam(lr=0.01),
             loss='binary_crossentropy',
             metrics=['acc'])

In [5]:
# ## custom loss
# from keras import backend as K
# import tensorflow as tf

# def focal_loss(gamma=2., alpha=.25):
#     def focal_loss_fixed(y_true, y_pred):
#         pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
#         pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
#         return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
#     return focal_loss_fixed
# ## cunstom metric

# def f1(y_true, y_pred):
#     def recall(y_true, y_pred):
#         """Recall metric.

#         Only computes a batch-wise average of recall.

#         Computes the recall, a metric for multi-label classification of
#         how many relevant items are selected.
#         """
#         true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#         possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#         recall = true_positives / (possible_positives + K.epsilon())
#         return recall

#     def precision(y_true, y_pred):
#         """Precision metric.

#         Only computes a batch-wise average of precision.

#         Computes the precision, a metric for multi-label classification of
#         how many selected items are relevant.
#         """
#         true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#         predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#         precision = true_positives / (predicted_positives + K.epsilon())
#         return precision
#     precision = precision(y_true, y_pred)
#     recall = recall(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))
# custom_loss = focal_loss()

In [6]:
# model.compile(optimizer = optimizers.Adam(lr=0.1),
#              loss= custom_loss,
#              metrics=['acc'])

In [7]:
## callback 함수를 쓴다면 추가
from keras.callbacks import EarlyStopping, ModelCheckpoint

cp = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
es = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1)

In [8]:
## 훈련시킨다.
model.fit(tt, train_y,
         epochs=100,
         validation_split=0.3,
         callbacks=[cp, es])


Train on 348 samples, validate on 150 samples
Epoch 1/100
348/348 [==============================] - 1s 2ms/step - loss: 0.5942 - acc: 0.6897 - val_loss: 0.5091 - val_acc: 0.7667
Epoch 2/100
348/348 [==============================] - ETA: 0s - loss: 0.4096 - acc: 0.843 - 0s 186us/step - loss: 0.4567 - acc: 0.7989 - val_loss: 0.4551 - val_acc: 0.8067
Epoch 3/100
348/348 [==============================] - 0s 166us/step - loss: 0.4188 - acc: 0.8247 - val_loss: 0.4422 - val_acc: 0.8000
Epoch 4/100
348/348 [==============================] - 0s 209us/step - loss: 0.4002 - acc: 0.8333 - val_loss: 0.4489 - val_acc: 0.8000
Epoch 5/100
348/348 [==============================] - 0s 195us/step - loss: 0.3837 - acc: 0.8563 - val_loss: 0.4168 - val_acc: 0.8067
Epoch 6/100
348/348 [==============================] - 0s 198us/step - loss: 0.3818 - acc: 0.8362 - val_loss: 0.4258 - val_acc: 0.8200
Epoch 7/100
348/348 [==============================] - 0s 201us/step - loss: 0.3674 - acc: 0.8362 - val_loss

In [9]:
## 모델 평가
res = model.predict(te)
def thres(a):
    if a < 0.5:
        return 0
    else:
        return 1
res = list(map(thres, res))
print(accuracy_score(test_y, res))

0.794392523364486


In [10]:
model.evaluate(te, test_y)

214/214 [==============================] - 0s 98us/step


[0.47600608423491503, 0.7943925261497498]

In [11]:
## 모형을 쌓듯이 하는 방법의 같은 코드
model = models.Sequential()

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))

model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer = optimizers.Adam(lr=0.1),
             loss='binary_crossentropy',
             metrics=['acc'])

model.fit(tt, train_y.values,
         epochs=40,
         validation_split=0.3,
         callbacks=[cp, es])


Train on 348 samples, validate on 150 samples
Epoch 1/40
348/348 [==============================] - 0s 507us/step - loss: 0.5309 - acc: 0.7529 - val_loss: 0.4841 - val_acc: 0.7600
Epoch 2/40
348/348 [==============================] - 0s 203us/step - loss: 0.4511 - acc: 0.7845 - val_loss: 0.4655 - val_acc: 0.7867
Epoch 3/40
348/348 [==============================] - 0s 172us/step - loss: 0.4117 - acc: 0.8161 - val_loss: 0.4952 - val_acc: 0.8000
Epoch 4/40
348/348 [==============================] - 0s 178us/step - loss: 0.4155 - acc: 0.8305 - val_loss: 0.5059 - val_acc: 0.8200
Epoch 5/40
348/348 [==============================] - 0s 186us/step - loss: 0.3922 - acc: 0.8132 - val_loss: 0.4778 - val_acc: 0.7800
Epoch 6/40
348/348 [==============================] - 0s 189us/step - loss: 0.4070 - acc: 0.8420 - val_loss: 0.5031 - val_acc: 0.8000
Epoch 7/40
348/348 [==============================] - 0s 199us/step - loss: 0.3990 - acc: 0.8305 - val_loss: 0.5799 - val_acc: 0.7533
Epoch 8/40
348/3